# Initialization

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from IPython.display import display, clear_output
from tqdm.auto import tqdm
import torch.nn.functional as F
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.profiler import profile, record_function, ProfilerActivity
import pickle
import os
import sys

ROOT = os.path.join("./")
sys.path.append(ROOT + "lib")
sys.path.append(ROOT + "models")

from helpers import *
from sourceset import SourceSet
from fourier import FourierModelTest, FourierModelTestNU
from final import AnomalyDetector
from transformer import FluxAnomalyPredictionTF
from general_trainer import compete, gen_train


device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_device(device)
torch.set_default_dtype(torch.float32)

/Users/matteo/.pyenv/versions/3.11.2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([[[ 0.,  1.,  2.],
         [ 3.,  4.,  5.],
         [ 6.,  7.,  8.],
         [ 9., 10., 11.],
         [12., 13., 14.],
         [15., 16., 17.],
         [18., 19., 20.],
         [21., 22., 23.],
         [24., 25., 26.],
         [27., 28., 29.]]])
tensor([[    0.,    60.,   336.,   990.,  2184.,  4080.,  6840., 10626., 15600.,
         21924.]])
tensor([[   60.,   990.,  4080., 10626., 21924.]])


In [2]:
# with open(ROOT + "processed_datasets/transittrain.pt", "rb") as f:
#   data_train = torch.load(f, map_location=device)
# with open(ROOT + "processed_datasets/transitvalid.pt", "rb") as f:
#   data_valid = torch.load(f, map_location=device)

with open(ROOT + "processed_datasets/freqtrain.pt", "rb") as f:
  data_train = torch.load(f, map_location=device)
with open(ROOT + "processed_datasets/freqtrain.pt", "rb") as f:
  data_valid = torch.load(f, map_location=device)



data_train, data_valid

(<sourceset.SourceSet at 0x104535c90>, <sourceset.SourceSet at 0x169dbb2d0>)

# Model

In [10]:
model = AnomalyDetector()
model_pars = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_pars])

for p in model.named_parameters():
    print(p[0])
    print(p[1].size())

print(params)

blend_bilinear.weight
torch.Size([6, 6, 6])
blend_bilinear.bias
torch.Size([6])
selfattn_signal.in_proj_weight
torch.Size([6, 2])
selfattn_signal.in_proj_bias
torch.Size([6])
selfattn_signal.out_proj.weight
torch.Size([2, 2])
selfattn_signal.out_proj.bias
torch.Size([2])
selfattn_fourier.in_proj_weight
torch.Size([6, 2])
selfattn_fourier.in_proj_bias
torch.Size([6])
selfattn_fourier.out_proj.weight
torch.Size([2, 2])
selfattn_fourier.out_proj.bias
torch.Size([2])
to_out.weight
torch.Size([3, 6])
to_out.bias
torch.Size([3])
291


In [17]:
train = DataLoader(data_train, batch_size=len(data_train)//100, shuffle=True, collate_fn=padded_collate)
valid = DataLoader(data_valid, batch_size=len(data_valid)//100, shuffle=False, collate_fn=padded_collate)

# Training Loop (Single)

In [18]:
kwargs=dict()
model = AnomalyDetector(**kwargs).to(device)
optim = torch.optim.Adam(model.parameters(), lr=0.003, weight_decay=0.00005)
loss = torch.nn.CrossEntropyLoss().to(device)
gen_train(model, kwargs, optim, loss, train, valid, 300, False)

Most Failed:  1
Epoch  22 :  0.9362206809222698
nulls: 0.8583333261805556 novas:  0.28333333097222224 pulsators:  0.24999997916666838 transits:  0.0


KeyboardInterrupt: 

In [ ]:
validaccessor = list(valid)[0]
idx = 8
ex = (validaccessor[0][idx], validaccessor[1][idx])
print(ex[1])
plot_from_tensor(ex[0])
# print(ex[0])

tensor([1., 0., 0.])


or

In [ ]:
samples = 256
lr = 0.007
dft = FourierModelTest(samples,2)
nudft = FourierModelTestNU(samples,2)

optim1 = torch.optim.Adam(dft.parameters(), lr=lr, weight_decay=0.0009)
optim2 = torch.optim.Adam(nudft.parameters(), lr=lr*0.7, weight_decay=0.0012)

compete([("dft", dft, optim1), ("nudft", nudft, optim2)], 300, train, valid)

Training Progress:   0%|          | 0/300 [00:00<?, ?it/s]

RuntimeError: 0D or 1D target tensor expected, multi-target not supported

In [14]:
model = AnomalyDetector().to(device)


with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
    with record_function("dd"):
        ex = next(iter(train))
        out = model(ex[0])
        # loss = loss_fn(out, ex[1])
        # loss.backward()
        # for name, param in model.cpu().named_parameters():
        #   print(name, param.grad)
        # plot_grad_flow(model.cpu().named_parameters())

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

/Users/matteo/.pyenv/versions/3.11.2/lib/python3.11/site-packages/torch/autograd/profiler.py:179: UserWarning:

CUDA is not available, disabling CUDA profiling

STAGE:2024-01-06 12:29:52 82338:5938829 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2024-01-06 12:30:05 82338:5938829 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2024-01-06 12:30:05 82338:5938829 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                     dd         0.78%     103.484ms       100.00%       13.214s       13.214s             1  
                                            aten::empty         0.00%      73.000us         0.00%      73.000us       3.476us            21  
                                          aten::random_         0.00%      10.000us         0.00%      10.000us       5.000us             2  
                                             aten::item         0.00%     157.000us         0.00%     158.000us      79.000us             2  
      